### Aim:
To solve the block world problem with the help of a* search and manhatton distance.


                                      RA1811027010097
                                        Jaya Durga V

### Procedure:
1. The input is initial state and goal state 
2. The goal is divided into sub goal states
3. With the help of manhatton distance , the heuristic vaule is calculated and a* searching algorithm is applied .
4. The goal state is reached with the least heuristic value and the output is printed with the number of moves. 

In [1]:
import time
from collections import deque


def __astar_heuristic(F, goal_layout, blocks_keys):
    
    scores = []     #A list object containing the score of each state.

    for state in F:
        score = 0     #Initialize each score to 0.

        for key in blocks_keys:     #For each block...
            if state.layout[key] != goal_layout[key]:     #If it is not in its final position...
                score += 1     #Add 1 to score.
        
        score += state.distance     #Add the distance from the root to score.
        
        scores.append(score)
    
    return scores.index(min(scores))     #Return the index of the state with the minimun score.


def heuristic_search(current_state, goal_state, method, timeout = 60):
    
    if method == 'astar':     #If the keyword 'astar' is passed, the algorithm becomes the A* Search algorithm.
        heuristic_function = __astar_heuristic

    F = []     #A list fot storing the nodes/states.
    discovered = set()     #A set for keeping the ids of the discovered states.
    blocks_keys = list(current_state.layout.keys())     #A list with the names/keys of the blocks.

    F.append(current_state)     #Add the current/initial state to the list.
    discovered.add(current_state.id)     #Add the id of the state to the set.

    st = time.perf_counter()     #Start a time counter.

    while F:     #While F is not empty...
        if time.perf_counter() - st > timeout:     #If the execution time exceeds the timeout...
            print('Timeout!')
            return None     #Break.

        i = heuristic_function(F, goal_state.layout, blocks_keys)     #Return the index of the state with the minimum score in F.
        state = F.pop(i)     #Pop the state with the minimum score.

        if state == goal_state:     #If the state is the goal state, return it and break.
            return state
        
        children = state.calcChildren()     #Else, calculate the children of this state.

        for child in children:     #For each child...
            if child.id not in discovered:     #If this child has not been discovered...
                discovered.add(child.id)     #Mark it as discovered.
                child.parent = state     #Set the parent attribute of the child to be the state that has been poped.

                F.append(child)     #Append the child to F.

In [2]:
from copy import deepcopy

class State:
    
    def __init__(self, layout, parent = None, move = [], distance = 0):
        self.layout = layout
        self.parent = parent
        self.move = move
        self.distance = distance

        values = list(self.layout.values())     #A list of the names of the blocks.

        self.id = ''.join([str(i) for s in values for i in s])     #Create the id attribute.
    
    def __eq__(self, other_state):
        """
        Override the build-in __eq__ method. Two states are equal if and only if they have
        the same id.
        """
        if other_state != None:
            return self.id == other_state.id
        else:
            return False

    def calcChildren(self):
        """
        The method creates a list of all the states that can be produced from a given state.
        It moves all the clear blocks to all available destinations and creates a new state 
        for each alteration.
        """
        layout = self.layout
        children = []

        free_blocks = [key for key in layout if layout[key][1] == 'c']     #The blocks that can be moved.

        for moving_block in free_blocks:     #For each free block that will be moved...
            for target_block in free_blocks:
                if moving_block != target_block:
                    temp = deepcopy(layout)     #Copy the current layout in order to alter it.
                    move = []
                    distance = 0

                    released_block = temp[moving_block][0]     #The 'released_block' is the first item of the list in layout with key == moving_block.

                    temp[moving_block][0] = target_block     #The 'moving block' now is on top of the 'target_block'.
                    temp[target_block][1] = 'u'     #And the 'target_block' is now unclear.

                    move.append(moving_block)     #Add the 'moving_block' to 'move' list.

                    if released_block != '-':     #If the 'released_block" is not '-' i.e. is not on the table...
                        temp[released_block][1] = 'c'     #Set the block clear.

                        move.append(released_block)     #Add the 'released_block' to 'move' list.
                    else:
                        move.append('table')
                    
                    move.append(target_block)     #Add the 'target_block' to 'move' list.
                    distance = self.distance + 1     #The distance of the child is the distance of the parent plus 1.

                    children.append(State(layout = temp, parent = self, move = move, distance = distance))     #Add to 'children' list a new State object.
            
            if layout[moving_block][0] != '-':     #If the 'moving_block' is not currently on the table, create a state that it is.
                temp = deepcopy(layout)
                move = []
                distance = 0

                released_block = temp[moving_block][0]     #The 'released_block' is the first item of the list in layout with key == moving_block.

                temp[moving_block][0] = '-'
                temp[released_block][1] = 'c'     #Set the block clear.

                move.append(moving_block)     #Add the 'moving_block' to 'move' list.
                move.append(released_block)      #Add the 'released_block' to 'move' list.
                move.append('table')

                distance = self.distance + 1     #The distance of the child is the distance of the parent plus 1.

                children.append(State(layout = temp, parent = self, move = move, distance = distance))     #Add to 'children' list a new State object.
        
        return children     #Return the children list.

In [6]:
def read_input_file(filename):
    """
    This function reads a problem file and returns two State objects, one for the initial
    state and one for the goal state.
    """
    with open('C:/Users/ADMIN/Documents/ai/lab/input.txt') as f:
        lines = [line.split() for line in f]     #Read the file by line and split it.

        blocks_names = lines[0][1:]     #Get the blocks names.
        blocks_names[-1] = blocks_names[-1][:-1]     #Remove the parenthesis from the last block name.

        initial = lines[1][1:-1]     #Get the initial state.
        initial = [i.replace('(', '') for i in initial]     #Remove the parentheses.
        initial = [i.replace(')', '') for i in initial]

        goal = lines[2][2:]     #Get the goal state.
        goal = [i.replace('(', '') for i in goal]     #Remove the parentheses.
        goal = [i.replace(')', '') for i in goal]

        initial_layout = {key: ['-', 'c'] for key in blocks_names}     #Construct the inital layout.
        goal_layout = {key: ['-', 'c'] for key in blocks_names}     #Construct the goal layout.

        for i in range(len(initial)):
            if initial[i] == 'ON':
                initial_layout[initial[i + 1]][0] = initial[i + 2]
                initial_layout[initial[i + 2]][1] = 'u'
        
        for i in range(len(goal)):
            if goal[i] == 'ON':
                goal_layout[goal[i + 1]][0] = goal[i + 2]
                goal_layout[goal[i + 2]][1] = 'u'

    return State(layout = initial_layout), State(layout = goal_layout)     #Return two state objects.

def write_output_file(solution, filename):
    """
    This function is taking as arguments a State object which is a solution of the problem,
    and a filename to write the steps towards the solution.
    """
    current_state = solution     #The state we start, which is the last i.e. the solution.
    path = []     #The path from the solution towards the intial state.
    i = 1
    
    while True:
        path.append(current_state)     #Add the current state i.e. solution, in the list.

        current_state = current_state.parent     #The current state now becomes the parent of it.
        
        if current_state.parent == None:     #If the current state has no parent...
            path.append(current_state)     #Add the current state in the list.
            break
    
    path.reverse()     #Reverse the list.
    
    with open('C:/Users/ADMIN/Documents/ai/lab/input.txt', 'r') as f:     #Open the output file.
        for state in path[1:]:     #For every state in the path, except the first one which is the initial state that has no previous move...
            move = state.move     #Get the move.

            print(str(i) + '. Move(' + move[0] + ', ' + move[1] + ', ' + move[2] + ')')     #Write the move.
            i += 1     #Increment the counter.
    
    return len(path[1:])     #Return the number of steps.

In [8]:
import time
import sys
import os

#from state import State
#from searching import breadth_first_search, depth_first_search, heuristic_search
#from utilities import read_input_file, write_output_file

def main():
    st = time.perf_counter()     #Start a time counter.
    method = 'astar'    
    initial_state, goal_state = read_input_file(filename = 'C:/Users/ADMIN/Documents/ai/lab/input.txt' )     #Read the input file and return two state objects.
    
    if method == 'astar':
        solution = heuristic_search(current_state = initial_state, goal_state = goal_state, method = 'astar', timeout = 300)
    else:     #If the method argument is none of the above, print a usage message.
        solution = None
        print('Usage: python bw.py <method> <input filename> <output filename>')

    if solution == goal_state:     #If the solution is equal to the goal state...
        number_of_moves = write_output_file(solution = solution, filename = 'C:/Users/ADMIN/Documents/ai/lab/input.txt')     #Write the solution file and return the number of moves.

        print('Solution found!')
        print('Number of blocks:', len(initial_state.layout.keys()))
        print('Method:', method)
        print('Number of moves:', number_of_moves)
        print('Execution time:', str(round(time.perf_counter() - st, 4)))

    else:     #Else, if the length of the keyword arguments is not equal to four, print a usage message.
        print('Usage: python bw.py <method> <input filename> <output filename>')

if __name__ == '__main__':
	main()

1. Move(E, D, table)
2. Move(C, B, table)
3. Move(D, table, C)
4. Move(B, A, table)
5. Move(A, table, E)
Solution found!
Number of blocks: 5
Method: astar
Number of moves: 5
Execution time: 0.0203


### Result :
The block world problem has been successfully executed with the a* algorithm and output is verified .